In [1]:
from algorithms.trace_the_best import TraceTheBest
import numpy as np
from util import utility_functions

In [2]:
ttb = TraceTheBest()

In [3]:
ttb.running_winner

[15]

In [4]:
ttb.actions

array([ 2,  8, 24, 25, 19, 11, 22,  7,  9, 17,  6, 13, 23,  3, 15])

In [5]:
ttb.subset

array([ 0,  1,  4,  5, 10, 12, 14, 16, 18, 20, 21, 26, 27, 28, 29])

In [6]:
ttb.step()

In [7]:
wins = dict()
for arm in ttb.actions:
    wins[arm] = ttb.preference_estimate.get_wins(arm_index=arm)

In [8]:
wins

{2: 2.0,
 8: 15.0,
 24: 21.0,
 25: 18.0,
 19: 20.0,
 11: 18.0,
 22: 23.0,
 7: 13.0,
 9: 12.0,
 17: 12.0,
 6: 16.0,
 13: 8.0,
 23: 10.0,
 3: 20.0,
 15: 2.0}

In [11]:
import csv 
import re
import os
from pathlib import Path
import numpy as np

running_times_file = os.path.join(f'{Path.cwd()}', 'Data_saps_swgcp_reduced\cpu_times_inst_param.csv')
running_times = []
with open(running_times_file, newline='') as csvfile:
    running_times_data = list(csv.reader(csvfile))
for i in range(1, len(running_times_data)):
    next_line = running_times_data[i][0]
    next_rt_vector = [float(s) for s in re.findall(r'-?\d+\.?\d*', next_line)][2:]
    running_times.append(next_rt_vector)
running_times = np.asarray(running_times)
lambda_ = 100
running_times = np.exp(-lambda_ * running_times)

In [13]:
running_times.shape

(5000, 20)

In [14]:
with open(os.path.join(f'{Path.cwd()}', 'Data_saps_swgcp_reduced\Random_Parameters_SAPS.txt'), "r") as f:
    print(f.readlines())

['[[1.5411388195908957, 0.8512124409204118, 0.7394411417919063, 0.8466405274871364], [1.8587190936898168, 0.6627008926747562, 0.5327588961637244, 0.19369267956194292], [1.488344679900774, 0.8393507549142253, -0.2195617150751633, 0.9736288384317074], [0.8078380961897467, 0.7878762538542863, 0.6349530938152467, 0.987748542571856], [1.2793716752186777, 0.7850202265117289, 0.8716241643012, 0.7690036238905535], [1.0879162357123897, 0.09994492756209727, 0.624429149132911, -0.2718023201831236], [1.6535073735760364, 0.8283015950731956, 0.7114344742221157, 0.9524186631101768], [1.451217193272738, 0.5158064487978745, 0.8046393481702985, -0.20273055153247843], [1.6245010199293852, 0.5537483410424786, 0.4642116795471889, 0.41949596087658847], [1.4036119873927375, 0.7281216123896138, 0.743331710855305, 0.7155452701265295], [1.929608951275947, 0.9107201115006488, 0.9952553739177741, 0.9582840403663512], [1.4095524115842906, 0.7408473379609312, -0.07884678392069745, -0.3678710459650717], [1.794813355

In [2]:
D:\Study Material\Thesis work\Multi-Dueling-Bandits\Data_saps_swgcp_reduced

'd:\\Study Material\\Thesis work\\Multi-Dueling-Bandits'